# 探索问题
1、年龄与逃生成功率的相关性

In [1]:
import numpy as np
import pandas as pd
import sys
import matplotlib as mpl 
import matplotlib.pyplot as plt
%matplotlib inline
titanic_file_name='titanic-data.csv'
df=pd.read_csv(titanic_file_name)
df_survived=df[df.Survived==1].copy()
#df[df.Survived==1].groupby('Age').plot.bar()

从表中看，名字，费用，登船地,票号对逃生结果无影响，删除这些列，但保存名字做后续分析
Cabin 的缺失数据很多，所以也删除

In [2]:
df.drop(['Embarked','Fare','Ticket','Cabin'],axis=1,inplace=True)

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0


主要分析性别与逃生的关系，确定有没有做到lady first


分析相关系数，从结果来看相关系数大于0.3小于0.8 是正相关，大致来看，女性逃生的概率大于男性，下面做详细分析

In [4]:
df['Survived'].corr(df['Sex']=='female')

0.54335138065775523

要排除其它因素影响

计算船上旅客性别的比例

In [5]:
total_female_proportion=(df[df['Sex']=='female']).shape[0]/(df.shape[0])

In [6]:
total_female_proportion

0.35241301907968575

In [7]:
survived_female_proportion=(df_survived[df_survived['Sex']=='female']).shape[0]/(df_survived.shape[0])

In [8]:
survived_female_proportion

0.6812865497076024

最后逃生的女性的比例大于 船上旅客的比例 

但是不排除这是女性的年龄段，船舱等级等因素造成的，继续分析

In [9]:
#####################################
# comput_proportion                 #
#####################################

## 从 DataFrame 中计算 col_2 对应的每个值占col_1的每个值的比例，返回比例构成的DF
def compute_proportion(df,col_1,col_2):
    unique_value_in_col_1=pd.unique(df.loc[:,col_1])
    unique_value_in_col_2=pd.unique(df.loc[:,col_2])
    proportion_df=pd.DataFrame(index=unique_value_in_col_1,columns=unique_value_in_col_2)
    for i in unique_value_in_col_1:
        for j in unique_value_in_col_2:
            col_1_count=((df[df.loc[:,col_1]==i]).count())['PassengerId']
            #col_2_count_meanwhile_col_1_count=(((df[df.loc[:,col_1]==i]).loc[:,col_2]==j).count())#['PassengerId']
            col_2_count_meanwhile_col_1_count=((df[df.loc[:,col_1]==i])[(df[df.loc[:,col_1]==i]).loc[:,col_2]==j]).count()['PassengerId']
            #print (col_1_count)
            #print ((df[df.loc[:,col_1]==i])[(df[df.loc[:,col_1]==i]).loc[:,col_2]==j])
            #print (col_2_count_meanwhile_col_1_count)
            proportion_df.loc[i,j]=col_2_count_meanwhile_col_1_count/col_1_count
    return proportion_df

In [10]:
compute_proportion(df,'Pclass','Sex')

,male,female
3,0.706721,0.293279
1,0.564815,0.435185
2,0.586957,0.413043


In [11]:
compute_proportion(df_survived,'Pclass','Sex')

,female,male
1,0.669118,0.330882
3,0.605042,0.394958
2,0.804598,0.195402


从结果来看三个等级的船舱中，登船旅客中女性的比例都要少于男性，但是幸存者中 三个船舱的女性幸存者的比例都要高于男性

下面继续分析年龄是否对我们的分析造成了干扰

In [12]:

df['age_group']='child'
df_survived['age_group']='child'

In [13]:
#####################################
# comput_proportion                 #
#####################################

## 从 DataFrame 中计算 age 对应的age_group,返回age_group series
def convert_age_group(age):
    if(age<=14):
        return 'child'
    if(age<=50):
        return 'adult'
    else:
        return 'senior citizen'

In [14]:
def convert_age(age_group):
    return age_group.map(convert_age_group)

In [15]:
df['age_group']=convert_age(df['Age'])
df_survived['age_group']=convert_age(df['Age'])

In [16]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,age_group
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,adult
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,adult
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,adult
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,adult
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,adult


# 先看Pclass==1

In [17]:
compute_proportion(df[df.Pclass==1],'age_group','Sex')

,female,male
adult,0.492958,0.507042
senior citizen,0.318841,0.681159
child,0.4,0.6


In [18]:
compute_proportion(df_survived[df_survived.Pclass==1],'age_group','Sex')

,female,male
adult,0.68,0.32
senior citizen,0.6875,0.3125
child,0.25,0.75


从上边两个表中来看，adult 和 senior citizen 都和之前的分析吻合  child却不同
下面分析Pclass中的child

In [19]:
df_child=df[df.age_group=='child']
df_child_pclass_1=df_child[df_child.Pclass==1]
df_child_pclass_1

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,age_group
297,298,0,1,"Allison, Miss. Helen Loraine",female,2.00,1,2,child
305,306,1,1,"Allison, Master. Hudson Trevor",male,0.92,1,2,child
435,436,1,1,"Carter, Miss. Lucile Polk",female,14.00,1,2,child
445,446,1,1,"Dodge, Master. Washington",male,4.00,0,2,child
802,803,1,1,"Carter, Master. William Thornton II",male,11.00,1,2,child


从结果来看，因为child的样本数比较小，而且存活率为80%，唯一一个去世的child 为female，
所以才导致了在Pclass ==1中 child 里 female的幸存都比例的降低

# Pclass==2

In [20]:
compute_proportion(df[df.Pclass==2],'age_group','Sex')

,female,male
child,0.526316,0.473684
senior citizen,0.192308,0.807692
adult,0.438849,0.561151


In [21]:
compute_proportion(df_survived[df_survived.Pclass==2],'age_group','Sex')

,female,male
child,0.526316,0.473684
senior citizen,0.571429,0.428571
adult,0.918033,0.0819672


Pclass==2 中旅客和幸存者中  child 的性别的比例是一样的，我们打印出来观察一下

In [22]:
df_child_pclass_2=df_child[df_child.Pclass==2]
df_child_pclass_2

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,age_group
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.00,1,0,child
43,44,1,2,"Laroche, Miss. Simonne Marie Anne Andree",female,3.00,1,2,child
58,59,1,2,"West, Miss. Constance Mirium",female,5.00,1,2,child
78,79,1,2,"Caldwell, Master. Alden Gates",male,0.83,0,2,child
183,184,1,2,"Becker, Master. Richard F",male,1.00,2,1,child
193,194,1,2,"Navratil, Master. Michel M",male,3.00,1,1,child
237,238,1,2,"Collyer, Miss. Marjorie ""Lottie""",female,8.00,0,2,child
340,341,1,2,"Navratil, Master. Edmond Roger",male,2.00,1,1,child
407,408,1,2,"Richards, Master. William Rowe",male,3.00,1,1,child
446,447,1,2,"Mellinger, Miss. Madeleine Violet",female,13.00,0,1,child


awesome!!! 所以的child都幸存了！！

# Pclass==3

In [23]:
compute_proportion(df[df.Pclass==3],'age_group','Sex')

,male,female
adult,0.743151,0.256849
senior citizen,0.705479,0.294521
child,0.509434,0.490566


In [25]:
compute_proportion(df_survived[df_survived.Pclass==3],'age_group','Sex')

,female,male
adult,0.532258,0.467742
child,0.590909,0.409091
senior citizen,0.742857,0.257143


pclass 3中的 幸存者中的女性幸存者的比例 也要高于整体旅客中的女性比例

In [26]:
df_pclass.get_group(1).head()

NameError: name 'df_pclass' is not defined

分别计算各个Pclass中female的比例

In [ ]:
df_plcass_1=df[df.Pclass==1]
df_plcass_2=df[df.Pclass==2]
df_plcass_3=df[df.Pclass==1]

In [ ]:
(df_plcass_1.count()/df.count())['PassengerId']
(df_plcass_2.count()/df.count())['PassengerId']
(df_plcass_3.count()/df.count())['PassengerId']

In [ ]:
df1_female_proportion=((df_plcass_1[df_plcass_1.Sex=='female'].count())/(df_plcass_1.count()))['PassengerId']
df2_female_proportion=((df_plcass_2[df_plcass_2.Sex=='female'].count())/(df_plcass_2.count()))['PassengerId']
df3_female_proportion=((df_plcass_3[df_plcass_3.Sex=='female'].count())/(df_plcass_3.count()))['PassengerId']

计算各个Pclass中的幸存者的female的比例

In [12]:
type(list(df.columns))
(list(df.columns))

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Cabin']

In [ ]:
unique_value_in_col_1=pd.unique(df.Sex)
unique_value_in_col_2=pd.unique(df.Pclass)
portion=pd.DataFrame(index=unique_value_in_col_2,columns=unique_value_in_col_1)
portion

In [ ]:
(portion.loc[3]).male

In [ ]:
pd.unique(df.loc[:,'Sex'])

In [ ]:
portion.loc[3,'male']

In [ ]:
comput_proportion(df,'Pclass','Sex')

In [ ]:
comput_proportion(df,'Sex','Pclass')

In [ ]:
df_pclass.get_group(1)

2、探索船舱号与逃生的相关性

In [ ]:
df[df.loc[:,'Cabin'].notnull()]
df[(df.loc[:,'Cabin'].notnull())&(df.loc[:,'Survived']==1)]

3、票价与逃生的关系

4、性别与逃生的关系

In [ ]:
df['Survived'].corr(df['Sex']=='female')

In [ ]:
df[df.Survived==1].groupby('Sex').sum()['Survived'].plot.bar()

In [ ]:
gb_sex=df.groupby('Sex')
gb_sex.get_group('male').PassengerId.count()

In [ ]:
gb_sex.get_group('female').PassengerId.count()